In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_count = pd.read_csv("../input/iwildcam2022-fgvc9/metadata/metadata/train_sequence_counts.csv")
df_count.head(10)

In [ ]:
count = []
for i in range(max(df_count["count"])):
    count.append(len(df_count[df_count["count"]==i]))
f = plt.figure(figsize=(15,8))
f = plt.title("distribution of count")
f = plt.bar([x for x in range(len(count))],count)

In [ ]:
import json, codecs

with codecs.open("../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_train_annotations.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
    
with codecs.open("../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_test_information.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    test_meta = json.load(f)

train_meta.keys()

In [ ]:
df_seq = pd.DataFrame(train_meta['images'])
df_seq = df_seq.drop(["seq_num_frames","datetime","width","width","height","id","sub_location","location"],axis=1)
df_seq.head(10)

# **Play a sequence**

In [ ]:
seq_id = "3021a08e-7d42-11eb-8fb5-0242ac1c0002"
df_c = df_count[df_count["seq_id"]==seq_id]
df_c

In [ ]:
df = df_seq[df_seq["seq_id"]==seq_id]
for i in range(len(df)):
    img = cv2.imread("../input/iwildcam2022-fgvc9/train/train/"+df.iloc[i]["file_name"])
    plt.imshow(img)
    plt.show()

# **Data setup**

In [ ]:
l_count,train,test = [],[],[]
for i in range(len(df_seq)):
    line = df_seq.iloc[i]
    seq_id = line["seq_id"]
    count = df_count[df_count["seq_id"]==seq_id]["count"]
    if len(count) == 1:
        train.append(line)
        l_count.append(count.values[0])
    else :
        pass
        #test.append(line)

In [ ]:
train = pd.DataFrame(train)
train["count"] = l_count

test = pd.DataFrame(test)
print("long",len(train))
train.head()

In [ ]:
# 16/9
#a = 20 *32
redim = [180,320]
redim

In [ ]:
X_train,y_train  = [],[]
for i in range(len(train)):
    line = train.iloc[i]
    path = line["file_name"]
    count = line["count"]
    y_train.append(count)
    img = cv2.imread("../input/iwildcam2022-fgvc9/train/train/"+path)
    X_train.append(cv2.resize(img, [redim[1],redim[0]],interpolation = cv2.INTER_AREA))
len(X_train),len(y_train)

In [ ]:
X_train[0].shape

In [ ]:
plt.imshow(X_train[50]);

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers

backbone = tf.keras.applications.mobilenet.MobileNet(
    input_shape=redim+[3],
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling=None,
)
backbone.trainable = False

In [ ]:
from tensorflow.keras.models import Model

inp = keras.Input(shape=redim+[3])
x = layers.BatchNormalization()(inp)
for layer in backbone.layers:
    x = layer(x)
x = layers.AveragePooling2D()(x)
x = layers.Conv2D(512,1)(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(256,activation="relu")(x)
x = layers.Dropout(.2)(x)
x = layers.Dense(1)(x)

model = Model(inp,x)
model.summary()

In [ ]:
model.compile(keras.optimizers.Adam(),loss=keras.losses.mae,metrics=keras.metrics.MAE)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=15, verbose=1,
    mode='auto', restore_best_weights=True
)

In [ ]:
h = model.fit(X_train,y_train,epochs=10,validation_split=.2,callbacks=[earlystop])

In [ ]:
# finetuning :
for layer in model.layers:
    layer.trainable = True
model.summary()

In [ ]:
ROP = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=0,
    mode="auto"
)

In [ ]:
model.compile(keras.optimizers.Adam(),loss=keras.losses.mae,metrics=keras.metrics.MAE)

In [ ]:
h = model.fit(X_train,y_train,epochs=10,validation_split=.2,callbacks=[earlystop,ROP])

In [ ]:
del X_train,y_train

# **Inferrance**

In [ ]:
import os 
test_path = os.listdir("../input/iwildcam2022-fgvc9/test/test")

In [ ]:
test_id = [path.split(".")[0] for path in test_path]
len(test_id)

In [ ]:
X_test = []
for path in test_path:
    img = cv2.imread("../input/iwildcam2022-fgvc9/test/test/"+path)
    X_test.append(cv2.resize(img, [redim[1],redim[0]],interpolation = cv2.INTER_AREA))
len(X_test)

In [ ]:
X_test = np.array(X_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred.shape